In [1]:
import psycopg2
import pandas.io.sql as sqlio
import pandas as pd
from downloader import Downloader
import numpy as np

conn = psycopg2.connect(dbname='diplom', user='diplomant', 
                        password='diplomant', host='localhost')

downloader = Downloader(conn)

In [2]:
tokens = downloader.get_tokens_list_DB().address.unique()
pools = downloader.get_pools_list_DB()

In [4]:
def inPool(address: str, pool: pd.Series) -> bool:
    return address==pool.token0_address["Address"] or address==pool.token1_address["Address"]

# [inPool(tokens[0], i) for _,i  in pools.iterrows()]
pools.last_reserve_update.iloc[0]['Liquidity']

'192784788986192922295026829369167891'

In [6]:
class Pool:
    def __init__ (self, addr, liq, res0, res1):
        self.address = addr
        self.liquidity = liq
        self.reserve0 = res0
        self.reserve1 = res1

    def __repr__(self)->str:
        return str(self.address+" "+self.liquidity+" "+ self.reserve0+" "+ self.reserve1)
        
class Pools:    
    def __init__ (self):
        self.pools = []
    
    def put(self, addr:str, liq:str, res0:str, res1:str):
        pool = Pool(addr, liq, res0, res1)
        self.pools.append(pool)
        
    def __repr__(self)->str:
        return str(len(self.pools))

    def zeroPool(self)->bool:
        return len(self.addresses)==0
    
    def __len__(self)->int:
        return len(self.addresses)
    
    def __getitem__(self, idx)->int:
        return self.pools[idx]
        
corr = pd.DataFrame(index=tokens, columns = tokens).fillna("undef")

for _,i  in pools.iterrows():
    token0 = i.token0_address['Address'] 
    token1 = i.token1_address['Address'] 
    address = i.address
    resrve0 = i.last_reserve_update['Reserve0']
    resrve1 = i.last_reserve_update['Reserve1']
    liq = i.last_reserve_update['Liquidity']
    if token0!= token1 and corr.loc[token0].loc[token1]=='undef':
        pool = Pools()
        pool.put(address,liq, resrve0, resrve1)
        corr.loc[token0].loc[token1] = pool
        corr.loc[token1].loc[token0] = pool
    else:
        corr.loc[token0].loc[token1].put(address,liq, resrve0, resrve1)

In [7]:
corr

,0xdAC17F958D2ee523a2206206994597C13D831ec7,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x4Fabb145d64652a948d72533023f6E7A623C7C53,0x6B175474E89094C44Da98b954EedeAC495271d0F,0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0,0xDad2a6Fb4c05dBa2924d9c88e9DAb828Fb3a0155,0x45804880De22913dAFE09f4980848ECE6EcbAf78,0x4d224452801ACEd8B2F0aebE155379bb5D594381,0xd0b9ed2C21849E830d6f6a9a7Ba6C1e952847569,...,0xeae17A7867F6BaBEABc8739bd34cb151Bf1C4A42,0x243cACb4D5fF6814AD668C3e225246efA886AD5a,0xf0f9D895aCa5c8678f706FB8216fa22957685A13,0x177B80D34FB77767E47B185692553e31782EC716,0x37cd4E8875E3EDafFDFe9Be63958f07eFfBD0Bfd,0xdAd33e12e61dC2f2692F2c12e6303B5Ade7277Ba,0x0C37Bcf456bC661C14D596683325623076D7e283,0xD13cfD3133239a3c73a9E535A5c4DadEE36b395c,0x9992eC3cF6A55b00978cdDF2b27BC6882d88D1eC,0x82EcdD4635766560c4e3A8EFa8a7C1Fff566111E
0xdAC17F958D2ee523a2206206994597C13D831ec7,undef,2,2,1,2,1,undef,1,2,undef,...,undef,undef,undef,undef,undef,undef,1,undef,undef,undef
0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2,undef,2,2,2,2,1,2,2,1,...,1,1,1,1,1,1,1,1,1,undef
0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,2,2,undef,1,2,2,undef,2,undef,undef,...,undef,undef,1,undef,undef,undef,undef,undef,1,undef
0x4Fabb145d64652a948d72533023f6E7A623C7C53,1,2,1,undef,1,undef,undef,undef,undef,undef,...,undef,undef,undef,undef,undef,undef,undef,undef,undef,undef
0x6B175474E89094C44Da98b954EedeAC495271d0F,2,2,2,1,undef,2,undef,1,undef,undef,...,undef,undef,undef,undef,undef,undef,undef,undef,undef,undef
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0xdAd33e12e61dC2f2692F2c12e6303B5Ade7277Ba,undef,1,undef,undef,undef,undef,undef,undef,undef,undef,...,undef,undef,undef,undef,undef,undef,undef,undef,undef,undef
0x0C37Bcf456bC661C14D596683325623076D7e283,1,1,undef,undef,undef,undef,undef,undef,undef,undef,...,undef,undef,undef,undef,undef,undef,undef,undef,undef,undef
0xD13cfD3133239a3c73a9E535A5c4DadEE36b395c,undef,1,undef,undef,undef,undef,undef,undef,undef,undef,...,undef,undef,undef,undef,undef,undef,undef,undef,undef,undef
0x9992eC3cF6A55b00978cdDF2b27BC6882d88D1eC,undef,1,1,undef,undef,undef,undef,undef,undef,undef,...,undef,undef,undef,undef,undef,undef,undef,undef,undef,undef
